In [ ]:
pip install openpyxl
dbutils.library.restartPython()
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import os
# Initialiser SparkSession avec le support Hive
spark = SparkSession.builder \
    .appName("Skytrax_Orly") \
    .enableHiveSupport() \
    .getOrCreate()
# Créer ou accéder à une table Hive pour stocker les chemins des fichiers traités
spark.sql("CREATE TABLE IF NOT EXISTS c_tech.skytrax_treated_files_orly (nom_fichier STRING)")
# Fonction pour ajouter les données à chaque DataFrame selon le périmètre
def add_data_to_df(subtheme, data, perimeter, theme, file_name):
    if perimeter == 'landside':
        # Sauter la première ligne et prendre les 5 premières colonnes
        temp_df = data.iloc[1:, :5].copy()
        temp_df.columns = ['Details', 'ORY1', 'ORY2', 'ORY3', 'ORY4']
    elif perimeter == 'airside':
        # Sauter la première ligne et prendre les 7 premières colonnes
        temp_df = data.iloc[1:, :7].copy()
        temp_df.columns = ['Details', 'Gates A', 'Gates B', 'Gates C', 'Gates D', 'Gates E', 'Gates F']
    elif perimeter == 'overall':
        # Sauter la première ligne et prendre les 2 premières colonnes
        temp_df = data.iloc[1:, :2].copy()
        temp_df.columns = ['Details', 'Rates']
    # Ajouter les colonnes communes
    temp_df['Subtheme'] = subtheme
    temp_df['Theme'] = theme
    temp_df['Plateforme'] = plateforme
    temp_df['Perimetre'] = perimeter
    temp_df['Year'] = year
    temp_df['Fichier_Source'] = file_name

    # Concaténer avec le DataFrame correspondant
    if perimeter == 'landside':
        global df_landside
        df_landside = pd.concat([df_landside, temp_df], ignore_index=True)
    elif perimeter == 'airside':
        global df_airside
        df_airside = pd.concat([df_airside, temp_df], ignore_index=True)
    elif perimeter == 'overall':
        global df_overall
        df_overall = pd.concat([df_overall, temp_df], ignore_index=True)
# Chemin du dossier contenant les fichiers CSV
dossier = '/dbfs/FileStore/Extime Patrimony/AERO/Raw/Skytrax/ORLY'

# Liste des noms de fichiers
file_names = [fichier for fichier in os.listdir(dossier) if fichier.endswith('.xlsx')]

# Identifier les nouveaux fichiers
# cette liste contiendra les noms de fichiers uniquement
nouveaux_fichiers = []
for file_name in file_names:
    # Vérifier si le nom du fichier est déjà dans la table Hive
    if not spark.sql(f"SELECT nom_fichier FROM c_tech.skytrax_treated_files_orly WHERE nom_fichier = '{file_name}'").collect():
        nouveaux_fichiers.append(file_name)
# Gestion des tables d'archivage
if nouveaux_fichiers:
    spark.sql("CREATE TABLE IF NOT EXISTS a_ing.Skytrax_ORY_Rating_landside LIKE c_ing.Skytrax_ORY_Rating_landside")
    spark.sql("CREATE TABLE IF NOT EXISTS a_ing.Skytrax_ORY_Rating_airside LIKE c_ing.Skytrax_ORY_Rating_airside")
    spark.sql("CREATE TABLE IF NOT EXISTS a_ing.Skytrax_ORY_Rating_overall LIKE c_ing.Skytrax_ORY_Rating_overall")

    spark.sql("CREATE TABLE IF NOT EXISTS a_ing.Skytrax_ORY_Rating_landside_overall LIKE c_ing.Skytrax_ORY_Rating_landside_overall")
    spark.sql("CREATE TABLE IF NOT EXISTS a_ing.Skytrax_ORY_Rating_airside_overall LIKE c_ing.Skytrax_ORY_Rating_airside_overall")
    spark.sql("CREATE TABLE IF NOT EXISTS a_ing.Skytrax_ORY_Rating_overall_overall LIKE c_ing.Skytrax_ORY_Rating_overall_overall")

    spark.sql("CREATE TABLE IF NOT EXISTS a_exp.Skytrax_Rating LIKE c_exp.Skytrax_Rating")

    spark.sql("CREATE TABLE IF NOT EXISTS a_rep.Skytrax_Rating LIKE c_rep.Skytrax_Rating")
else:
    print("Aucun nouveau  fichier dans cette repertoire, veuillez réessayer utlterièrement")
if nouveaux_fichiers:
   # Insertion des anciens tables dans l'archive
 spark.sql("""
      INSERT INTO a_ing.Skytrax_ORY_Rating_landside
      SELECT * FROM c_ing.Skytrax_ORY_Rating_landside
      where Year NOT IN (SELECT Year FROM a_ing.Skytrax_ORY_Rating_landside)
    """)
 spark.sql("""
      INSERT INTO a_ing.Skytrax_ORY_Rating_airside
      SELECT * FROM c_ing.Skytrax_ORY_Rating_airside
      where Year NOT IN (SELECT Year FROM a_ing.Skytrax_ORY_Rating_airside)
   """)
 spark.sql("""
      INSERT INTO a_ing.Skytrax_ORY_Rating_overall
      SELECT * FROM c_ing.Skytrax_ORY_Rating_overall
      where Year NOT IN (SELECT Year FROM a_ing.Skytrax_ORY_Rating_overall)
   """)
 spark.sql("""
      INSERT INTO a_ing.Skytrax_ORY_Rating_landside_overall
      SELECT * FROM c_ing.skytrax_ory_rating_landside_overall
      where Year NOT IN (SELECT Year FROM a_ing.Skytrax_ORY_Rating_landside_overall)
   """)
 spark.sql("""
      INSERT INTO a_ing.Skytrax_ORY_Rating_airside_overall
      SELECT * FROM c_ing.Skytrax_ORY_Rating_airside_overall
      where Year NOT IN (SELECT Year FROM a_ing.Skytrax_ORY_Rating_airside_overall)
   """)
 spark.sql("""
      INSERT INTO a_ing.Skytrax_ORY_Rating_overall_overall
      SELECT * FROM c_ing.Skytrax_ORY_Rating_overall_overall
      where Year NOT IN (SELECT Year FROM a_ing.Skytrax_ORY_Rating_overall_overall)
   """)
 spark.sql("""
      INSERT INTO a_exp.Skytrax_Rating
      SELECT * FROM c_exp.Skytrax_Rating
      where Year NOT IN (SELECT Year FROM a_exp.Skytrax_Rating) AND Plateforme NOT IN (SELECT Plateforme FROM a_exp.Skytrax_Rating)
   """)
 spark.sql("""
      INSERT INTO a_rep.Skytrax_Rating
      SELECT * FROM c_rep.Skytrax_Rating
      where Year NOT IN (SELECT Year FROM a_rep.Skytrax_Rating) AND Plateforme NOT IN (SELECT Plateforme FROM a_rep.Skytrax_Rating)
   """)
else:
   print("Aucun nouveau  fichier dans cette repertoire, veuillez réessayer utlterièrement")
if nouveaux_fichiers:
    for nom_fichier in nouveaux_fichiers:
     # Reconstruire le chemin complet
     file_path = os.path.join(dossier, nom_fichier)

     file_name= os.path.basename(file_path)
     skytrax = pd.ExcelFile(file_path)

     parts = file_name.split('_')

     plateforme = parts[0]
     year = parts[3]

     if plateforme == "ORY":
          plateforme = "ORLY"

     # Preparation de Table Row

     # Traitement de l'onglet Overall
     for sheet_name in skytrax.sheet_names:
         if sheet_name == "Overall":
            # Lire l'onglet dans un DataFrame
             df = skytrax.parse(sheet_name,header=None)
             df = df.dropna(axis=0, how='all')
             df = df.reset_index(drop=True)

             theme = sheet_name

             # Initialisation des listes pour stocker les indices de lignes
             five_columns_indices = []
             seven_columns_indices = []
             two_columns_indices = []

             # Trouver les indices des lignes contenant "Aggregate" pour délimiter les tables
             aggregate_indices = df.index[df[0].str.contains("Aggregate", na=False)].tolist()

             # Initialiser un index de début à 0 pour la première table
             start_index = 0

             # Listes pour stocker les DataFrames de chaque table
             tables = []

             for end_index in aggregate_indices:
                 # Extraire chaque table jusqu'à "Aggregate" inclus
                 table = df.iloc[start_index:end_index+1].reset_index(drop=True)
                 tables.append(table)

                 # Mettre à jour le start_index pour la prochaine table
                 start_index = end_index + 1  # +1 pour exclure la ligne aggregate pour le debut du prochain table

             df_five_columns= tables[0]
             df_seven_columns= tables[1]
             df_two_columns= tables[2]

             df_five_columns = df_five_columns.drop(df_five_columns.columns[5:7], axis=1)
             df_two_columns = df_two_columns.drop(df_two_columns.columns[2:7], axis=1)

             # Renommer les colonnes
             df_five_columns.columns = ['Details', 'ORY1', 'ORY2', 'ORY3', 'ORY4']

             # Sauter la première ligne et prendre jusqu'à l'avant-dernière ligne
             df_landside_overall = df_five_columns.iloc[1:-1].reset_index(drop=True)

             # Renommer les colonnes
             df_seven_columns.columns = ['Details', 'Gates A', 'Gates B', 'Gates C', 'Gates D', 'Gates E', 'Gates F']

             # Sauter la première ligne et prendre jusqu'à l'avant-dernière ligne
             df_airside_overall = df_seven_columns.iloc[1:-1].reset_index(drop=True)

             # Renommer les colonnes
             df_two_columns.columns = ['Details', 'Rates']

             # Sauter la première ligne et prendre jusqu'à l'avant-dernière ligne
             df_overall_overall = df_two_columns.iloc[2:-1].reset_index(drop=True)

             df_landside_overall['Theme'] = theme
             df_landside_overall['Plateforme'] = plateforme
             df_landside_overall['Perimetre'] = 'landside'
             df_landside_overall['Year'] = year
             df_landside_overall['Fichier_Source'] = file_name

             df_airside_overall['Theme'] = theme
             df_airside_overall['Plateforme'] = plateforme
             df_airside_overall['Perimetre'] = 'Airside'
             df_airside_overall['Year'] = year
             df_airside_overall['Fichier_Source'] = file_name

             df_overall_overall['Theme'] = theme
             df_overall_overall['Plateforme'] = plateforme
             df_overall_overall['Perimetre'] = 'Overall'
             df_overall_overall['Year'] = year
             df_overall_overall['Fichier_Source'] = file_name

             landside_subthemes = list(df_landside_overall['Details'])
             landside_subthemes = [mot.strip() for mot in landside_subthemes]
             airside_subthemes = list(df_airside_overall['Details'])
             airside_subthemes = [mot.strip() for mot in airside_subthemes]
             overall_subthemes = list(df_overall_overall['Details'])
             overall_subthemes = [mot.strip() for mot in overall_subthemes]

      # Initialisation des DataFrames pour chaque périmètre
     df_landside = pd.DataFrame(columns=['Details', 'ORY1', 'ORY2', 'ORY3', 'ORY4', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year', 'Fichier_Source'])
     df_airside = pd.DataFrame(columns=['Details', 'Gates A', 'Gates B', 'Gates C', 'Gates D', 'Gates E', 'Gates F', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year', 'Fichier_Source'])
     df_overall = pd.DataFrame(columns=['Details', 'Rates', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year', 'Fichier_Source'])

     # Traitement des tous les autres onglets
     for sheet_name in skytrax.sheet_names:
          if sheet_name != "Overall":
              # Lire l'onglet dans un DataFrame
             df = skytrax.parse(sheet_name,header=None)
             df = df.dropna(axis=0, how='all')
             df = df.reset_index(drop=True)

             theme = sheet_name

             # Identifier tous les subthèmes présents dans l'onglet en respectant la structure corrigée
             subthemes = df.iloc[:, 0].dropna().unique()

             # Nouvelle identification du périmètre de chaque subthème, en tenant compte de la correction
             subthemes_perimeter_corrected = {
             "landside": [],
             "airside": [],
             "overall": []
             }

             for subtheme in subthemes:
                  # Vérifier si le subthème correspond aux catégories connues, sinon l'ajouter à "overall"
                 if subtheme in landside_subthemes:
                     subthemes_perimeter_corrected["landside"].append(subtheme)
                 elif subtheme in airside_subthemes:
                     subthemes_perimeter_corrected["airside"].append(subtheme)
                 elif subtheme in overall_subthemes:
                     subthemes_perimeter_corrected["overall"].append(subtheme)

             # Fusionner tous les sous-thèmes pour simplifier l'extraction
             all_subthemes = subthemes_perimeter_corrected['landside'] + subthemes_perimeter_corrected['airside'] + subthemes_perimeter_corrected['overall']

             # Réinitialisation du dictionnaire pour stocker les données correctement extraites pour chaque sous-thème
             subtheme_contents = {}
             current_subtheme = None
             start_index = None

             for index, row in df.iterrows():
                 # Détection d'un sous-thème
                 if row[0] in all_subthemes:
                     if current_subtheme is not None:
                          # Extraction du contenu du sous-thème précédent jusqu'à "Aggregate Rating"
                         end_index = df.index[(df[0] == "Aggregate Rating") & (df.index > start_index)].min()
                         subtheme_contents[current_subtheme] = df.iloc[start_index:end_index].reset_index(drop=True)
                     current_subtheme = row[0]
                     start_index = index + 1
                 elif index == df.shape[0] - 1 and current_subtheme is not None:
                     #Gérer le dernier sous-thème
                     end_index = df.index[(df[0] == "Aggregate Rating") & (df.index > start_index)].min()
                     subtheme_contents[current_subtheme] = df.iloc[start_index:end_index].reset_index(drop=True)
             # Appliquer la fonction à chaque sous-thème
             for subtheme, data in subtheme_contents.items():
                 if subtheme in subthemes_perimeter_corrected['landside']:
                     add_data_to_df(subtheme, data, 'landside', theme, file_name)
                 elif subtheme in subthemes_perimeter_corrected['airside']:
                     add_data_to_df(subtheme, data, 'airside', theme, file_name)
                 elif subtheme in subthemes_perimeter_corrected['overall']:
                     add_data_to_df(subtheme, data, 'overall', theme, file_name)


     ################################################################## Table Row
     df_landside_raw = df_landside
     df_airside_raw = df_airside
     df_overall_raw = df_overall

     df_landside_overall_raw = df_landside_overall
     df_airside_overall_raw = df_airside_overall
     df_overall_overall_raw = df_overall_overall

     # Conversion des colonnes en numérique
     df_landside_raw[['ORY1', 'ORY2', 'ORY3', 'ORY4']] = df_landside_raw[['ORY1', 'ORY2', 'ORY3', 'ORY4']].apply(pd.to_numeric, errors='coerce')
     df_airside_raw[['Gates A', 'Gates B', 'Gates C', 'Gates D','Gates E','Gates F']] = df_airside_raw[['Gates A', 'Gates B', 'Gates C', 'Gates D','Gates E','Gates F']].apply(pd.to_numeric, errors='coerce')
     df_overall_raw[['Rates']] = df_overall_raw[['Rates']].apply(pd.to_numeric, errors='coerce')

     # Conversion des colonnes en numérique
     df_landside_overall_raw[['ORY1', 'ORY2', 'ORY3', 'ORY4']] = df_landside_overall_raw[['ORY1', 'ORY2', 'ORY3', 'ORY4']].apply(pd.to_numeric, errors='coerce')
     df_airside_overall_raw[['Gates A', 'Gates B', 'Gates C', 'Gates D','Gates E','Gates F']] = df_airside_overall_raw[['Gates A', 'Gates B', 'Gates C', 'Gates D','Gates E','Gates F']].apply(pd.to_numeric, errors='coerce')
     df_overall_overall_raw[['Rates']] = df_overall_overall_raw[['Rates']].apply(pd.to_numeric, errors='coerce')

     # Convertir le DataFrame Pandas en DataFrame Spark
     sdf_landside_raw = spark.createDataFrame(df_landside_raw)
     sdf_airside_raw = spark.createDataFrame(df_airside_raw)
     sdf_overall_raw = spark.createDataFrame(df_overall_raw)

     sdf_landside_overall_raw = spark.createDataFrame(df_landside_overall_raw)
     sdf_airside_overall_raw = spark.createDataFrame(df_airside_overall_raw)
     sdf_overall_overall_raw = spark.createDataFrame(df_overall_overall_raw)

     for col in sdf_airside_raw.columns:
         new_col_name = col.replace(" ", "")
         sdf_airside_raw = sdf_airside_raw.withColumnRenamed(col, new_col_name)

     for col in sdf_airside_overall_raw.columns:
         new_col_name = col.replace(" ", "")
         sdf_airside_overall_raw = sdf_airside_overall_raw.withColumnRenamed(col, new_col_name)

     # Ecrire le DataFrame Spark dans Hive
     sdf_landside_raw.write.mode("append").saveAsTable("c_ing.Skytrax_ORY_Rating_landside")
     sdf_airside_raw.write.mode("append").saveAsTable("c_ing.Skytrax_ORY_Rating_airside")
     sdf_overall_raw.write.mode("append").saveAsTable("c_ing.Skytrax_ORY_Rating_overall")

     sdf_landside_overall_raw.write.mode("append").saveAsTable("c_ing.Skytrax_ORY_Rating_landside_overall")
     sdf_airside_overall_raw.write.mode("append").saveAsTable("c_ing.Skytrax_ORY_Rating_airside_overall")
     sdf_overall_overall_raw.write.mode("append").saveAsTable("c_ing.Skytrax_ORY_Rating_overall_overall")

     ########################################################################################## Table Exp
     # Utilisation de melt pour transformer les colonnes en lignes
     df_landside_exp = pd.melt(df_landside_raw, id_vars=['Details', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year'],
                    value_vars=['ORY1', 'ORY2', 'ORY3', 'ORY4'],
                    var_name='Terminal', value_name='Note_Terminal')

     # Reorganiser les colonnes
     df_landside_exp = df_landside_exp[['Details', 'Note_Terminal', 'Terminal', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year']]

     # Utilisation de melt pour transformer les colonnes en lignes
     df_airside_exp = pd.melt(df_airside_raw, id_vars=['Details', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year'],
                    value_vars=['Gates A', 'Gates B', 'Gates C', 'Gates D','Gates E','Gates F'],
                    var_name='Porte', value_name='Note_Porte')

     # Reorganiser les colonnes
     df_airside_exp = df_airside_exp[['Details', 'Note_Porte', 'Porte', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year']]

     df_overall_exp = df_overall_raw[['Details', 'Rates', 'Subtheme', 'Theme', 'Plateforme', 'Perimetre', 'Year']]

      # Ajout des colonnes manquantes pour uniformiser les schémas de chaque DataFrame
     df_landside_exp['Porte'] = np.nan
     df_landside_exp['Note_Porte'] = np.nan
     df_landside_exp['Rates'] = np.nan

     df_airside_exp['Terminal'] = np.nan
     df_airside_exp['Note_Terminal'] = np.nan
     df_airside_exp['Rates'] = np.nan

     df_overall_exp['Terminal'] = np.nan
     df_overall_exp['Note_Terminal'] = np.nan
     df_overall_exp['Porte'] = np.nan
     df_overall_exp['Note_Porte'] = np.nan

     # Concaténer les DataFrames préparés verticalement
     df_exp = pd.concat([df_landside_exp, df_airside_exp, df_overall_exp], ignore_index=True)

     # Réorganiser les colonnes si nécessaire
     columns_order = ['Details','Subtheme', 'Theme', 'Terminal', 'Note_Terminal', 'Porte', 'Note_Porte', 'Rates', 'Plateforme', 'Perimetre', 'Year']
     df_exp = df_exp[columns_order]

     # Convertir le DataFrame Pandas en DataFrame Spark
     sdf_exp = spark.createDataFrame(df_exp)

     # Ecrire le DataFrame Spark dans Hive
     sdf_exp.write.mode("append").saveAsTable("c_exp.Skytrax_Rating")

     ############################################################################### Table View

     df_view = df_exp.drop([df_exp.columns[0],df_exp.columns[2]], axis=1)

     df_landside_view = df_view.groupby(['Subtheme','Terminal']).agg({
     'Note_Terminal': 'mean',
     'Plateforme': 'first',
     'Perimetre': 'first',
     'Year': 'first'
     }).reset_index()

     # Arrondir à un dixième près
     df_landside_view['Note_Terminal'] = df_landside_view['Note_Terminal'].round(1)

     df_airside_view = df_view.groupby(['Subtheme','Porte']).agg({
     'Note_Porte': 'mean',
     'Plateforme': 'first',
     'Perimetre': 'first',
     'Year': 'first'
     }).reset_index()

     # Arrondir à un dixième près
     df_airside_view['Note_Porte'] = df_airside_view['Note_Porte'].round(1)

     #Filtrer pour inclure les lignes ou Perimetre est overall
     df_overall_view_filtered = df_view[df_view['Perimetre']=='overall']

     df_overall_view = df_overall_view_filtered .groupby('Subtheme').agg({
     'Rates': 'mean',
     'Plateforme': 'first',
     'Perimetre': 'first',
     'Year': 'first'
     }).reset_index()

     # Arrondir à un dixième près
     df_overall_view['Rates'] = df_overall_view['Rates'].round(1)

     # Ajout des colonnes manquantes pour uniformiser les schémas de chaque DataFrame
     df_landside_view['Porte'] = np.nan
     df_landside_view['Note_Porte'] = np.nan
     df_landside_view['Rates'] = np.nan

     df_airside_view['Terminal'] = np.nan
     df_airside_view['Note_Terminal'] = np.nan
     df_airside_view['Rates'] = np.nan

     df_overall_view['Terminal'] = np.nan
     df_overall_view['Note_Terminal'] = np.nan
     df_overall_view['Porte'] = np.nan
     df_overall_view['Note_Porte'] = np.nan

     # Concaténer les DataFrames préparés verticalement
     df_view_finale = pd.concat([df_landside_view, df_airside_view, df_overall_view], ignore_index=True)
     # Réorganiser les colonnes si nécessaire
     columns_order = ['Subtheme', 'Terminal', 'Note_Terminal', 'Porte', 'Note_Porte', 'Rates', 'Plateforme', 'Perimetre', 'Year']
     df_view_finale = df_view_finale[columns_order]

     # Convertir le DataFrame Pandas en DataFrame Spark
     sdf_view = spark.createDataFrame(df_view_finale)

     # Ecrire le DataFrame Spark dans Hive
     sdf_view.write.mode("append").saveAsTable("c_rep.Skytrax_Rating")

     ################################################################################################## MAJ de la table des nouveaux fichiers

     # Enregistrer les chemins des nouveaux fichiers traités dans Hive
     spark.sql(f"INSERT INTO c_tech.skytrax_treated_files_orly VALUES ('{file_name}')")
else:
    print("Aucun nouveau  fichier dans cette repertoire, veuillez réessayer utlterièrement")
%sql
select * from c_tech.skytrax_treated_files_orly
%sql
select * from c_rep.Skytrax_Rating
%sql
select count(*) from c_rep.Skytrax_Rating where Year="2024"
%sql
select count(*) from c_rep.Skytrax_Rating where Year="2021"
%sql
select count(*) from c_rep.Skytrax_Rating where Year="2022"
%sql
select count(*) from c_rep.Skytrax_Rating where Year="2021" and Porte is not null
%sql
select distinct(Subtheme) from c_rep.Skytrax_Rating where Year="2021" and Terminal is not null order by Subtheme asc